# Votorantim - Teste para a vaga Engenheiro de Dados

#### Objetivo:
Realizar o ETL dos dados do arquivo [TESTE]hubway.sqlite.db para o banco de dados Azure SQL

O Conjunto de dados possui 2 tabelas, stations e trips. Os dados de cada uma dessas tabelas sera analisado e quaisquer problemas/inconsistencias serao corrigidos.

#### Primeiro passo: carregamento dos dados

Os dados estão no formato sqlite, por isso, sera necessario o uso da biblioteca sqlite3 para o python3
O ETL poderia ser inteiro feito apenas com querys em SQL, entretanto, ele sera feito com o Pandas porque o sqlite tem suporte limitado a RegEx - por exemplo REGEX_SUB

ref: https://stackoverflow.com/questions/38877856/replace-a-part-of-a-string-with-regexp-in-sqlite3

In [1]:
# sqlite3 para carregar dos dados do arquivo [TESTE]hubway.sqlite.db
import sqlite3
# pandas para analsies e ETL
import pandas as pd
# connector para - necessario para o pandas poder carregar os dados em um df
conn = sqlite3.connect('[TESTE]hubway.sqlite.db')
# cursor para fazer qualquer query diretamente no banco de dados
c = conn.cursor()

In [2]:
# Esse banco de dados foi explorado com um extensão SQLITE para o VS CODE
# com isso foi possivel ver de quais tabelas e dados esse conjunto possui
# sao duas tabelas, stations e trips
# as duas serao carregadas em dataframes com uma query do tipo SELECT ALL
stations_df = pd.read_sql_query("SELECT * FROM 'stations'", conn)
trips_df = pd.read_sql_query("SELECT * FROM 'trips'", conn)

#### Analise dos dados em stations

In [3]:
# primeiro, melhor ter certeza de que todos os dados estao no dataframe
print(f'Para estacoes, no sqlite temos {len(stations_df)} entradas')
c.execute("SELECT count(*) FROM 'stations'")
print(f'Para estacoes, no dataframe temos {c.fetchone()[0]} entradas')

Para estacoes, no sqlite temos 142 entradas
Para estacoes, no dataframe temos 142 entradas


In [4]:
# Melhor dar uma olhada nesses dados antes de continuar
stations_df.head()

,id,station,municipality,lat,lng
0,3,Colleges%%%of%%%the%%%%Fenway,Boston,42.340021,-71.100812
1,4,Tremont St. at Berkeley St.,Boston,42.345392,-71.069616
2,5,Northeastern U / North Parking Lot,Boston,42.341814,-71.090179
3,6,Cambridge St. at Joy St.,Boston,42.361284999999995,-71.06514
4,7,Fan******Pier,Boston,42.353412,-71.044624


Aqui vemos um problema nesses dados, algumas estacoes estao com caracteres especiais no lugar dos espacos, isso pode causar alguns problemas no futuro, por exemplo, talvez alguem queira fazer algum join com o nome da cidade como chave

In [5]:
# algumas das estacoes com problemas nos nomes
print(stations_df['station'][0])
print(stations_df['station'][4])
print(stations_df['station'][16])

Colleges%%%of%%%the%%%%Fenway
Fan******Pier
Buswell$$$$$$$$$$$Park


Existem diversas maneiras de resolver o problema, mas aqui isso será resolvido em 2 etapas:
1. Com uma RegEx, substituir alguns caracteres especiais por espacos em branco
2. Com outra RegEx, substituir, onde houver, varios espacos por apenas 1

In [6]:
# 1 - substituir *, $ e % por espacos
stations_df = stations_df.replace(r'[<*$%]', ' ', regex=True)

In [7]:
print(stations_df['station'][0])
print(stations_df['station'][4])
print(stations_df['station'][16])

Colleges   of   the    Fenway
Fan      Pier
Buswell           Park


In [8]:
# 2 - substituir varios espacos, onde houver, por apenas 1 espaco
import re
# novo campo com o nome das estacoes corrigidos chama-se station_name
stations_df['station_name'] = stations_df.apply(lambda row: re.sub(' +', ' ', row.station), axis=1)

In [9]:
print('Nomes de estacoes corrigidos :')
print()
print(stations_df['station_name'][0])
print(stations_df['station_name'][4])
print(stations_df['station_name'][16])

Nomes de estacoes corrigidos :

Colleges of the Fenway
Fan Pier
Buswell Park


Obs: Essas duas regex poderiam ser feitas de uma vez so, foram feitas separadas para deixar claro o racional do processo

In [10]:
stations_df = stations_df.drop(columns=['station_name'])
stations_df.head()

,id,station,municipality,lat,lng
0,3,Colleges of the Fenway,Boston,42.340021,-71.100812
1,4,Tremont St. at Berkeley St.,Boston,42.345392,-71.069616
2,5,Northeastern U / North Parking Lot,Boston,42.341814,-71.090179
3,6,Cambridge St. at Joy St.,Boston,42.361284999999995,-71.06514
4,7,Fan Pier,Boston,42.353412,-71.044624


In [11]:
# cria arquivo csv
stations_df.to_csv('stations.csv')

#### Analise dos dados em Trips

In [12]:
print(f'Para trips, no sqlite temos {len(trips_df)} entradas')
c.execute("SELECT count(*) FROM 'trips'")
print(f'Para estacoes, no dataframe temos {c.fetchone()[0]} entradas')

Para trips, no sqlite temos 1570001 entradas
Para estacoes, no dataframe temos 1570001 entradas


In [13]:
trips_df.head()

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,1,9,1311847920,23,1311847920,23,B00468,Registered,'97217,1976,Male
1,2,220,1311848460,23,1311848700,23,B00554,Alterar para Registered,'02215,1966,Male
2,3,56,1311849180,23,1311849240,23,B00456,Alterar para Registered,'02108,1943,Male
3,4,64,1311849300,23,1311849360,23,B00554,Registered,'02116,1981,Female
4,5,12,1311849420,23,1311849420,23,B00554,Registered,'97214,1983,Female


In [14]:
# Primeiro vamos analisar os campo gender
# vamos ver quais dados sao encontrados nesses campos
print(trips_df.gender.unique())

['Male' 'Female' '' 'deletar' 'Female\n']


Esse 'deletar' esta fora do dominio e precisa ser corrigido
temos alguams opcoes:
1. deixar em branco 
2. substituir pelo mais comum (M,F)
3. deletar, como o proprio nome sugere
4. outros

Essas entradas serao deletadas

In [15]:
print(f"{len(trips_df[trips_df.gender == 'deletar'])} entradas serao excluidas")

13095 entradas serao excluidas


In [16]:
trips_df = trips_df[trips_df.gender != 'deletar']

In [17]:
print(trips_df.gender.unique())

['Male' 'Female' '' 'Female\n']


Male e Female fazem parte do dominio gender, mas Female/n e '' nao. Campos vazios podem ocorre caso esse campo nao seja obrigatorio por exemplo.

Campos com Female/n precisam ser transformados em Female, isso pode ser feito de varias formas, aqui vamos usar RegEx novamente para retirar todos os '\n' de Female

In [18]:
trips_df = trips_df.replace(r'[\n]', '', regex=True)

In [19]:
print(trips_df.gender.unique())

['Male' 'Female' '']


Agora temos apenas Male, Female e campos em branco.
Vamos analisar agora o campo sub_type

In [20]:
print(trips_df.sub_type.unique())

['Registered' 'Alterar para Registered' 'Casual']


Os campos com 'Alterar para Registered' precisam ser transformados para 'Registered'

In [21]:
trips_df.loc[trips_df['sub_type'] == 'Alterar para Registered', 'sub_type'] = 'Registered'

In [22]:
print(trips_df.sub_type.unique())

['Registered' 'Casual']


Agora trips esta com todos os problemas encontrados corrigidos

In [23]:
trips_df.head()

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,1,9,1311847920,23,1311847920,23,B00468,Registered,'97217,1976,Male
1,2,220,1311848460,23,1311848700,23,B00554,Registered,'02215,1966,Male
2,3,56,1311849180,23,1311849240,23,B00456,Registered,'02108,1943,Male
3,4,64,1311849300,23,1311849360,23,B00554,Registered,'02116,1981,Female
4,5,12,1311849420,23,1311849420,23,B00554,Registered,'97214,1983,Female


In [24]:
# cria arquivo csv
trips_df.to_csv('trips.csv')

### Merging Data

para fins de analise, seria interessante ter dados da estacao e da viagem no mesmo conjunto de dados. Precisaremos dar 2 merges (joins), serao 2 left joins para adicionar os dados do destino e da origem ao conjunto de dados viagens (trips).
Com isso seria possivel fazer analises mais interessante, por exemplo, qual municipalidade possui viagens mais longes? Viagens intermunicipais sao comuns?

Para isso, vamos criar 2 dataframes diferntes (dest_station_def e org_station_def) - copias de stations -  e mudar o nome de suas colunas 

In [25]:
dest_station_df = stations_df
dest_station_df.columns = ['dest_id', 'dest_station', 'dest_municipality', 'dest_lat', 'dest_lng']
merged_data = trips_df.merge(dest_station_df, left_on='end_station', right_on='dest_id', how='left')

In [26]:
org_station_df = stations_df
org_station_df.columns = ['org_id', 'org_station', 'org_municipality', 'org_lat', 'org_lng']
merged_data = merged_data.merge(dest_station_df, left_on='start_station', right_on='org_id', how='left')

In [27]:
# Para termos certeza de que o merge ocorreu de forma correta
# merged_data deve ter a mesma quantidade de entradas que trips_df
len(trips_df)-len(merged_data)

0

In [28]:
merged_data.head()

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,...,dest_id,dest_station,dest_municipality,dest_lat,dest_lng,org_id,org_station,org_municipality,org_lat,org_lng
0,1,9,1311847920,23,1311847920,23,B00468,Registered,'97217,1976,...,23.0,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364,23.0,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364
1,2,220,1311848460,23,1311848700,23,B00554,Registered,'02215,1966,...,23.0,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364,23.0,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364
2,3,56,1311849180,23,1311849240,23,B00456,Registered,'02108,1943,...,23.0,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364,23.0,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364
3,4,64,1311849300,23,1311849360,23,B00554,Registered,'02116,1981,...,23.0,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364,23.0,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364
4,5,12,1311849420,23,1311849420,23,B00554,Registered,'97214,1983,...,23.0,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364,23.0,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364


In [29]:
merged_data.to_csv('merged_data.csv')

Agora ficou bem mais facil responder algumas questoes, por exemplo: Qual a propocao de viagens que comecam em uma municipalidade e acabam em outra (intermunicipais)?

In [30]:
ratio = len(merged_data[merged_data.org_municipality != merged_data.dest_municipality])/len(merged_data)

print(f'De todas as viagens, {round(ratio,2)*100}% delas são entre municipalidades')

De todas as viagens, 20.0% delas são entre municipalidades
